In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_519740/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_9_39,0.991676,0.588148,0.980293,0.989349,0.986183,0.002726,0.244493,0.005792,0.005801,0.005796,0.021912,0.052213,0.982659,0.052632,141.809704,326.168028,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_9_40,0.991660,0.585164,0.980254,0.988296,0.985485,0.002731,0.246265,0.005804,0.006374,0.006089,0.021833,0.052263,0.982625,0.052683,141.805872,326.164196,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_9_38,0.991560,0.593476,0.980598,0.989658,0.986490,0.002764,0.241330,0.005703,0.005633,0.005668,0.021892,0.052575,0.982417,0.052997,141.782066,326.140390,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_9_41,0.991538,0.583669,0.980187,0.987294,0.984811,0.002771,0.247152,0.005823,0.006920,0.006372,0.021811,0.052644,0.982371,0.053067,141.776794,326.135118,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_9_37,0.991522,0.594225,0.980592,0.989669,0.986495,0.002777,0.240886,0.005704,0.005627,0.005665,0.021947,0.052694,0.982338,0.053117,141.773038,326.131362,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.577119,0.333437,0.668200,0.517989,0.581836,0.138501,0.395701,0.075562,0.150724,0.113143,0.198344,0.372157,0.118999,0.375146,133.953761,318.312085,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.437658,0.213466,0.532766,0.353471,0.429773,0.184177,0.466921,0.106405,0.202169,0.154287,0.236510,0.429158,-0.171546,0.432605,133.383720,317.742043,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.189631,0.040494,0.278124,0.133428,0.195458,0.265410,0.569604,0.164396,0.270976,0.217686,0.279715,0.515179,-0.688269,0.519317,132.652961,317.011285,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.088763,0.010125,0.221560,0.051458,0.124288,0.298446,0.587633,0.177278,0.296608,0.236943,0.292498,0.546302,-0.898411,0.550690,132.418334,316.776658,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
